## Give me the best deck with Champion X

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'./../data/raw/lor_ranked_data_all.csv') # read in the data

In [7]:
champion = 'Diana'
date = '2022-08-01'
n = 50

In [8]:
def best_deck_by_champ(df, champion, date, n):
    df_date = df[df['game_start_time_utc'] > date]
    champ_list = df_date[df_date['archetype'].str.contains(champion)]
    archetypes = champ_list['deck_code'].value_counts() # find the unique archetypes being played
    archetypes = archetypes[archetypes > n] #only keep popular deck_codes
    popular_list = list(archetypes.index) # make a list of popular deck_codes
    champ_list = champ_list.loc[df['deck_code'].isin(popular_list)] # filter df to only keep popular archetypes
    df_champ = champ_list.groupby(['deck_code', 'game_outcome']).size()
    df_champ = df_champ.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)*100)
    df_champ.drop('loss', level=1, axis=0, inplace=True) #drop loss %
    df_champ = df_champ.droplevel(level=1) #drop win/loss
    df_champ = df_champ.sort_values(ascending=False)
    
    df_small = df_date[['deck_code', 'archetype']]
    y = pd.DataFrame(df_champ)
    
    inner = pd.merge(y, df_small, on='deck_code', how='inner')#.value_counts()
    inner = inner.value_counts()
    df_champ = inner.sort_values(ascending=False)
    
    return df_champ

In [9]:
best_deck_by_champ(df, champion, date, n) # sorted by playrate

deck_code                                                                                        0     archetype          
CICQCAIFF4AQIBILAEDAKEADAMCQGBIGAQBQSIZYJHLACAYBAMCQEAIGBETAEAYJLFPAEAIDAUCACBQFDY               43.0  Diana Nocturne         136
CICQCAIFAEAQGBICAECAKCYBAYESMCIDBERTQSKLJ5NVZWAB3EAQAAIBAMEVK                                    39.0  Diana Aphelios (SI)     97
CICACBAFBMAQMCJGAMBQKAYFAYDQGCJDHBEVQWOWAHMACAQBAECS6AIDAUBAA                                    57.0  Diana Nocturne          89
CICQCAIFF4AQIBILAEDAKEADAMCQGBIGAQBQSIZYJHLACAQBAMEVSAIDAUBAKAIBAUMQCAYFAQAQMCJGAEDAKHQCAMEVKXQ  45.0  Diana Nocturne          80
dtype: int64

In [10]:
def best_deck_by_champ_by_winrate():
    x = best_deck_by_champ(df, champion, date, n)
    df_x = pd.DataFrame(x)
    df_x.index.names = ["deck_code", "winrate", "archetype"]
    df_x.reset_index(level=['winrate', 'archetype'], inplace=True)
    df_x = df_x.sort_values(by=['winrate'], ascending=False)
    return df_x
best_deck_by_champ_by_winrate()

,winrate,archetype,0
deck_code,,,
CICACBAFBMAQMCJGAMBQKAYFAYDQGCJDHBEVQWOWAHMACAQBAECS6AIDAUBAA,57.0,Diana Nocturne,89
CICQCAIFF4AQIBILAEDAKEADAMCQGBIGAQBQSIZYJHLACAQBAMEVSAIDAUBAKAIBAUMQCAYFAQAQMCJGAEDAKHQCAMEVKXQ,45.0,Diana Nocturne,80
CICQCAIFF4AQIBILAEDAKEADAMCQGBIGAQBQSIZYJHLACAYBAMCQEAIGBETAEAYJLFPAEAIDAUCACBQFDY,43.0,Diana Nocturne,136
CICQCAIFAEAQGBICAECAKCYBAYESMCIDBERTQSKLJ5NVZWAB3EAQAAIBAMEVK,39.0,Diana Aphelios (SI),97


In [110]:
# # double check it worked
# df_date = df[df['game_start_time_utc'] > date] #build dataframe you build in function
# deck = y.index[0][0] # get deck code returned for deck #1 in the function
# true_value = df_date.deck_code.str.count(deck).sum() #see how many times that deck actually appears in dataframe
# output_value = y[0] # get how many times the function said that deck appears
# true_value == output_value # compare values

In [65]:
## write code to stalk someone

In [66]:
contain_values = df[df['player'].str.contains('J Tiller#NA1', na=False)]
contain_values['archetype'].value_counts()

Lissandra Taliyah       89
Diana Aphelios (SI)     28
Akshan Kai'Sa (DE)      22
Trundle Gnar (PZ)       21
Renekton Akshan (BC)    20
Ashe LeBlanc            15
Hecarim Gwen (SH)        9
Thresh Nasus             5
Renekton Gwen            5
Fizz Riven (BC)          4
Ekko Sivir Zilean        3
Name: archetype, dtype: int64

In [10]:
contain_values = df[df['player'].str.contains('Majiin', na=False)]
contain_values['archetype'].value_counts()
# contain_values

Ahri Bard                  62
Ezreal Annie               56
Jarvan IV Poppy Bard       50
Twisted Fate Nami (SI)     45
Aphelios Viktor            11
Ezreal Kennen (IO)          9
Zoe Aphelios Yuumi (SI)     6
Elise Annie                 2
Tryndamere Trundle (IO)     2
Katarina Ahri               1
Name: archetype, dtype: int64

In [11]:
contain_values = df[df['player'].str.contains('Griffener#1217', na=False)]
contain_values['archetype'].value_counts()

Taric Poppy (DE)             7
Rumble Sion (BC)             4
Jarvan IV Illaoi             4
Jarvan IV Poppy Bard         2
Pyke Rek'Sai                 2
Malphite Taliyah             1
Katarina Ahri Annie          1
Sivir Akshan (DE)            1
Ashe LeBlanc                 1
Yuumi Poppy Tristana (DE)    1
Name: archetype, dtype: int64

In [12]:
contain_values = df[df['player'].str.contains('HEIF', na=False)]
contain_values['archetype'].value_counts()

Lissandra Taliyah                  16
Annie Jhin                          5
Miss Fortune Twisted Fate Annie     3
Irelia Azir                         2
Elise Annie                         1
Ahri Bard                           1
Vi Zoe Aphelios                     1
Name: archetype, dtype: int64

In [ ]:
##

In [15]:
def personal_winrate(user):
    contain_values = df[df['player'].str.contains('HEIF', na=False)]
    # contain_values['archetype'].value_counts()
    return contain_values['archetype'].value_counts()

In [16]:
personal_winrate('HEIF')

Lissandra Taliyah                  16
Annie Jhin                          5
Miss Fortune Twisted Fate Annie     3
Irelia Azir                         2
Elise Annie                         1
Ahri Bard                           1
Vi Zoe Aphelios                     1
Name: archetype, dtype: int64

In [20]:
contain_values = df[df['player'].str.contains('HEIF', na=False)]
contain_values.head(2)

,Unnamed: 0,match_id,game_start_time_utc,game_version,player,deck_code,game_outcome,order_of_play,total_turn_count,faction_1,faction_2,archetype,is_master,region
101,101,d5d9bc45-657d-41aa-8481-dddab8cfec1f,2022-06-14 18:40:14,live_3_09_12,HEIFER#OCE,CEDACAQDBEAQGBIQAECAGAQCAEBS4NYCAECSQNICAYBQYH...,loss,1,20,Noxus,ShadowIsles,Elise Annie,0,americas
334,334,46de58d1-c62c-4a2a-91da-71fe84bf7f8a,2022-06-14 16:57:03,live_3_09_12,HEIFER#OCE,CMCACBQBDYAQMBZSAQCACAIFBIHAMBAHBUPSYQSJPABACA...,win,0,18,Freljord,Shurima,Lissandra Taliyah,0,americas


In [29]:
outcomes = contain_values.groupby(['archetype', 'game_outcome']).size()
outcomes

archetype                        game_outcome
Ahri Bard                        win             1
Annie Jhin                       loss            1
                                 win             4
Elise Annie                      loss            1
Irelia Azir                      loss            2
Lissandra Taliyah                loss            9
                                 win             7
Miss Fortune Twisted Fate Annie  loss            1
                                 win             2
Vi Zoe Aphelios                  loss            1
dtype: int64

In [47]:
# sum win%
# outcomes

s = contain_values.groupby(['game_outcome'])['archetype'].value_counts()
print(s)

# df_small = df_date[['deck_code', 'archetype']]
# y = pd.DataFrame(df_champ)

game_outcome  archetype                      
loss          Lissandra Taliyah                  9
              Irelia Azir                        2
              Annie Jhin                         1
              Elise Annie                        1
              Miss Fortune Twisted Fate Annie    1
              Vi Zoe Aphelios                    1
win           Lissandra Taliyah                  7
              Annie Jhin                         4
              Miss Fortune Twisted Fate Annie    2
              Ahri Bard                          1
Name: archetype, dtype: int64


In [125]:
# contain_values = df[df['player'].str.contains('HEIF', na=False)]

wins = sum(contain_values['game_outcome'].str.contains('win', na=False))
loss = sum(contain_values['game_outcome'].str.contains('loss', na=False))

print('You win', round(wins/len(contain_values)*100, 2),'% of the time')

You win 48.28 % of the time


In [67]:
contain_values.index

334

In [27]:
outcomes = outcomes.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)*100)
outcomes

archetype                        game_outcome
Ahri Bard                        win             100.0
Annie Jhin                       loss             20.0
                                 win              80.0
Elise Annie                      loss            100.0
Irelia Azir                      loss            100.0
Lissandra Taliyah                loss             56.0
                                 win              44.0
Miss Fortune Twisted Fate Annie  loss             33.0
                                 win              67.0
Vi Zoe Aphelios                  loss            100.0
dtype: float64

In [69]:
outcomes.drop('loss',level=1, axis=0, inplace=True)
outcomes = outcomes.droplevel(level=1)
outcomes = outcomes.sort_values(ascending=False)
outcomes

archetype
Lissandra Taliyah                  7
Annie Jhin                         4
Miss Fortune Twisted Fate Annie    2
Ahri Bard                          1
dtype: int64

In [70]:
outcomes.index[1]

'Annie Jhin'

In [ ]:
##

In [118]:
contain_values.head(2)

,Unnamed: 0,match_id,game_start_time_utc,game_version,player,deck_code,game_outcome,order_of_play,total_turn_count,faction_1,faction_2,archetype,is_master,region
463565,463565,e3c0736e-9e2e-457d-971e-69a8f4ebccb1,2022-06-23 00:30:18,live_3_09_12,J Tiller#NA1,CQDACAIABEAQGAAOAECAAAYBAUFB2AYDBENSGMYDAUEQGB...,win,0,26,Demacia,MtTargon,Yuumi Pantheon (DE),0,americas
586470,586470,0643dd07-f631-4049-a01e-819a89385b92,2022-06-25 17:24:22,live_3_09_12,J Tiller#NA1,CMCQCAIBGIAQMAI6AEDAOMQDAQAQCBIOAMCAOH2CJEBACA...,win,0,28,Freljord,Shurima,Lissandra Taliyah,0,americas


In [122]:
contain_values = df[df['player'].str.contains('HEIF', na=False)]
outcomes = contain_values.groupby(['archetype', 'game_outcome']).size()
outcomes = outcomes.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)*100)
# outcomes
outcomes.drop('loss',level=1, axis=0, inplace=True)
outcomes = outcomes.droplevel(level=1)
outcomes = outcomes.sort_values(ascending=False)
outcomes

archetype
Ahri Bard                          100.0
Annie Jhin                          80.0
Miss Fortune Twisted Fate Annie     67.0
Lissandra Taliyah                   44.0
dtype: float64

In [124]:
y = pd.DataFrame(outcomes)
y['n_games'] = 0
_ = []
for i in outcomes.index:
    print(i)
    y['n_games'][i] = sum(contain_values['archetype'].str.contains(str(i), na=False))
    _.append(sum(contain_values['archetype'].str.contains(str(i), na=False)))
    # y['n_games'].iloc[:, i] = sum(contain_values['archetype'].str.contains(str(i), na=False))
y
# _

Ahri Bard
Annie Jhin
Miss Fortune Twisted Fate Annie
Lissandra Taliyah


<ipython-input-124-f2edfdc13181>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['n_games'][i] = sum(contain_values['archetype'].str.contains(str(i), na=False))


,0,n_games
archetype,,
Ahri Bard,100.0,1
Annie Jhin,80.0,5
Miss Fortune Twisted Fate Annie,67.0,3
Lissandra Taliyah,44.0,16


In [ ]:
## get win% of archetypes

In [110]:
contain_values = df[df['player'].str.contains('J Tiller#NA1', na=False)]
outcomes = contain_values.groupby(['archetype', 'game_outcome']).size()
outcomes = outcomes.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)*100)
outcomes.drop('loss',level=1, axis=0, inplace=True)
outcomes = outcomes.droplevel(level=1)
outcomes = outcomes.sort_values(ascending=False)
outcomes

archetype
Elise Viego (NX)        100.0
Yuumi Pantheon (DE)     100.0
Fizz Riven (BC)          75.0
Lissandra Taliyah        67.0
Diana Aphelios (SI)      39.0
Renekton Akshan (BC)     35.0
Ekko Sivir Zilean        33.0
Trundle Gnar (PZ)        25.0
Thresh Nasus             20.0
dtype: float64

In [78]:
outcomes.index[1]

'Annie Jhin'

In [ ]:
## get n for archetype

In [79]:
for i in outcomes.index:
    print(i)

Ahri Bard
Annie Jhin
Miss Fortune Twisted Fate Annie
Lissandra Taliyah


In [85]:
sum(contain_values['archetype'].str.contains('Ahri Bard', na=False))

1

In [111]:
y = pd.DataFrame(outcomes)
y['n_games'] = 0
for i in outcomes.index:
    y['n_games'][i] = sum(contain_values['archetype'].str.contains(str(i), na=False))
    # y['n_games'].iloc[:, i] = sum(contain_values['archetype'].str.contains(str(i), na=False))
y

<ipython-input-111-a8f14b1fc63e>:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  y['n_games'][i] = sum(contain_values['archetype'].str.contains(str(i), na=False))
<ipython-input-111-a8f14b1fc63e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['n_games'][i] = sum(contain_values['archetype'].str.contains(str(i), na=False))


,0,n_games
archetype,,
Elise Viego (NX),100.0,0
Yuumi Pantheon (DE),100.0,0
Fizz Riven (BC),75.0,0
Lissandra Taliyah,67.0,30
Diana Aphelios (SI),39.0,0
Renekton Akshan (BC),35.0,0
Ekko Sivir Zilean,33.0,3
Trundle Gnar (PZ),25.0,0
Thresh Nasus,20.0,5


In [105]:
y

,0,n_games
archetype,,
Ahri Bard,100.0,1
Annie Jhin,80.0,5
Miss Fortune Twisted Fate Annie,67.0,3
Lissandra Taliyah,44.0,16


In [94]:
outcomes.add(game_count) #lmao adds it to the existing numbers

archetype
Ahri Bard                          101.0
Annie Jhin                          85.0
Miss Fortune Twisted Fate Annie     70.0
Lissandra Taliyah                   60.0
dtype: float64

In [95]:
outcomes

archetype
Ahri Bard                          100.0
Annie Jhin                          80.0
Miss Fortune Twisted Fate Annie     67.0
Lissandra Taliyah                   44.0
dtype: float64

In [97]:
y = pd.DataFrame(outcomes)

In [99]:
inner = pd.merge(y, outcomes, how='inner')

ValueError: Cannot merge a Series without a name

In [ ]:
    df_small = df_date[['deck_code', 'archetype']]
    y = pd.DataFrame(df_champ)
    
    inner = pd.merge(y, df_small, on='deck_code', how='inner')#.value_counts()
    inner = inner.value_counts()
    df_champ = inner.sort_values(ascending=False)
    
    return df_champ